In [3]:
%pip install --user gym
%pip install pygame --pre
%pip install gym[box2d]
%pip install numpy
%pip install -U matplotlib
%pip install pynput
%pip install keyboard


import gym
from gym import wrappers
import matplotlib
import matplotlib.pyplot as plt
import keyboard
import time
import datetime as dt
from datetime import datetime
    
main_loop_time = 0.03 # measured in seconds

keys_being_pressed = {"w": False, "a": False, "s": False, "d": False}

def choosePlayerActionDiscrete(): # choose ONE action based on keys_being_pressed 
    
    if keys_being_pressed['s']: # brake pressed
        return 4 # brake
    elif keys_being_pressed['w']: # brake off, gas pressed
        return 3 # gas
    else: #just steering 
        if (keys_being_pressed['a'] and keys_being_pressed['d']) or not(keys_being_pressed['a'] or keys_being_pressed['d']): # both or neither
            return 0 # do nothing, steering cancels out
        elif keys_being_pressed['a']: # turn left ... these are backwards???
            return 2
        else: # turn right
            return 1


#target env, will create env object ok
env = gym.make("CarRacing-v2", render_mode='human', continuous=False)

# If continuous = False, actions are:
# 0: do nothing
# 1: steer left   (pretty sure this is actually steer right and vice versa)
# 2: steer right
# 3: gas
# 4: brake

# If continuous = True, actions are:
# of type numpy.ndarray[] (length 3) 
# three floats: 
# index 0: steering, -1 is full left, +1 is full right
# index 1: gas
# index 2: braking

env.reset()

end_loop = False
max_iterations = 100000 #aaa
i = 0

while (i < max_iterations and not end_loop): #main gameplay loop
    i += 1

    for key in ['w', 'a', 's', 'd']:
        keys_being_pressed[key] = keyboard.is_pressed(key)

    if (keyboard.is_pressed('q')):
        end_loop = True

    start_time = datetime.now()
    target_end_time = start_time + dt.timedelta(seconds=main_loop_time)
    
    while datetime.now() < target_end_time: # wait for loop time
        pass

    #print("passed")

    #show image
    env.render()
    #action = env.action_space.sample()
    action = choosePlayerActionDiscrete()
    #print(str(type(action)))
    #print("action: " + str(action))
    env.step(action)
env.close()

print("finished")

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\nickp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\nickp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


...

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\nickp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0


C:\Users\nickp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 0
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
action: 3
<class 'int'>
ac